In [ ]:
import sys
sys.path.append('/home/b/bechtol/repos/ts_config_ocs')

In [ ]:
from rubin_scheduler.data import get_data_dir

In [ ]:
at_usdf = True
if at_usdf:
    #os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/group/rubin/shared/rubin_sim_data"
    os.environ["RUBIN_SIM_DATA_DIR"] = "/home/b/bechtol/rubin-user/rubin_sim_data"
print("Using rubin_sim_data from ", get_data_dir())

In [ ]:
"""
import numpy as np
import rubin_scheduler.scheduler.basis_functions as bf
import rubin_scheduler.scheduler.detailers as detailers
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.schedulers import CoreScheduler, ComCamFilterSched
from rubin_scheduler.scheduler.surveys import FieldSurvey
from rubin_scheduler.scheduler.utils import Footprint, SkyAreaGenerator
from rubin_scheduler.utils import SURVEY_START_MJD as MJD_START

from astropy.time import Time
"""
from rubin_scheduler.scheduler.schedulers import CoreScheduler

In [ ]:
from Scheduler.feature_scheduler.maintel.fbs_config_sit_survey_block_t248 import gen_field_survey

In [ ]:
nside = 32

filters_list = [
    #"i_06",
    #"r_03"
    "i",
    "r",
]

surveys = []
i = 0
for offset in range(0, 30):
    surveys.append(gen_field_survey(f'BLOCK-T248_{i}', sequence=filters_list))
    i += 1
scheduler = CoreScheduler(surveys, nside=nside)

In [ ]:
scheduler.survey_lists

In [ ]:
print(scheduler.survey_lists[0][0].science_program)
print(scheduler.survey_lists[0][0].survey_name)
print(scheduler.survey_lists[0][0].target_name)
print(scheduler.survey_lists[0][0].observation_reason)
print(scheduler.survey_lists[0][0].scheduler_note)

In [ ]:
scheduler.survey_lists[0][0].basis_functions

In [ ]:
#dir(Scheduler.feature_scheduler.maintel.fbs_config_sit_survey_block_t248)

In [ ]:
import numpy as np
from astropy.time import Time

from rubin_scheduler.scheduler.schedulers import ComCamFilterSched
from rubin_scheduler.scheduler.model_observatory import ModelObservatory, KinemModel
from rubin_scheduler.site_models import ConstantSeeingData, ConstantWindData
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.utils import SchemaConverter

In [ ]:
def tma_movement(percent=70):
    # See https://confluence.lsstcorp.org/display/LSSTCOM/TMA+Motion+Settings
    # Expected performance at end of comcam on-sky is probably 10%
    if percent > 125:
        percent = 125
        print("Cannot exceed 125 percent, by requirements.")
    tma = {}
    scale = percent / 100.0
    tma['azimuth_maxspeed'] = np.min([10.0 * scale, 7.0])
    tma['azimuth_accel'] = 10.0 * scale
    tma['azimuth_jerk'] = np.max([1.0, 40.0 * scale])
    tma['altitude_maxspeed'] = 5.0 * scale
    tma['altitude_accel'] = 5.0 * scale
    tma['altitude_jerk'] = np.max([1.0, 20.0 * scale])
    tma['settle_time'] = 3.0
    return tma

In [ ]:
def rotator_movement(percent=100):
    # Kevin and Brian say these can run 100% and are independent of TMA movement
    if percent > 125:
        percent = 125
        print("Cannot exceed 125 percent, by requirements.")
    rot = {}
    rot['maxspeed'] = 3.5 * percent/100
    rot['accel'] = 1.0 * percent/100
    rot['jerk'] = 4.0 * percent/100
    return rot

In [ ]:
def get_model_observatory(dayobs='2024-11-01'):
    # Set up a fresh model observatory
    survey_start = Time(f'{dayobs}T12:00:00', format='isot', scale='utc')
    mjd_now = survey_start.mjd

    kinematic_model = KinemModel(mjd0=mjd_now)
    kinematic_model.setup_camera(readtime=2.4, **rotator_movement(100.))
    kinematic_model.setup_telescope(**tma_movement(10.0))
    
    # Some weather telemetry that might be useful
    seeing_data = ConstantSeeingData(fwhm_500=2.0)
    wind_data = ConstantWindData(wind_direction=340, wind_speed=5.0)
    
    # Set up the model observatory
    observatory = ModelObservatory(mjd=mjd_now, 
                                   mjd_start=survey_start.mjd,
                                   kinem_model=kinematic_model, # Modified kinematics
                                   cloud_data='ideal',          # No clouds
                                   seeing_data=seeing_data,     # Modified seeing
                                   wind_data=wind_data,         # Add some wind
                                   downtimes='ideal',           # No downtime
                                   lax_dome=True,               # dome crawl?
                                   init_load_length=1,          # size of skybrightness files to load first
                                  )
    return observatory

In [ ]:
def update_model_observatory_sunset(observatory, filter_scheduler):
    # Make sure correct filters are mounted
    conditions = observatory.return_conditions()
    filters_needed = filter_scheduler(conditions)
    observatory.observatory.mount_filters(filters_needed)
    conditions = observatory.return_conditions()
    print("Current filter set", conditions.mounted_filters)
    
    # Move ahead to twilight
    observatory.mjd = conditions.sun_n18_setting
    print("Current observatory time", observatory.mjd)
    return observatory

In [ ]:
verbose = False

dayobs = '2024-11-03'
survey_start = Time(f'{dayobs}T12:00:00', format='isot', scale='utc')
mjd_now = survey_start.mjd

# Set up again to be sure all is 'clean'
#scheduler, filter_scheduler = get_comcam_schedulers()
#filter_scheduler = ComCamFilterSched()
filter_scheduler = ComCamFilterSched(
    #loaded_filter_groups=(('r', 'i'),),
    loaded_filter_groups=(filters_list,),
    illum_bins=(0, 100)
)
observatory = get_model_observatory(dayobs)
observatory = update_model_observatory_sunset(observatory, filter_scheduler)
conditions = observatory.return_conditions()

mjd = observatory.mjd
mjd_end = conditions.sun_n12_rising
sim_duration = mjd_end - mjd

print(Time(mjd, format='mjd').iso)


observatory, scheduler, observations = sim_runner(
    observatory,
    scheduler,
    filter_scheduler=filter_scheduler,
    sim_start_mjd=mjd,
    sim_duration=sim_duration,
    n_visit_limit=None,
    step_none=10.0,
    verbose=True,
    record_rewards=False,
    start_result_size=int(2e4),
    append_result_size=int(2.5e4),
)

In [ ]:
sc = SchemaConverter()
visits = sc.obs2opsim(observations)

In [ ]:
visits.columns

In [ ]:
sorted(np.unique(visits['target_name']))

In [ ]:
visits[['target_name', 'filter']][0:60]

In [ ]:
observations

In [ ]:
scheduler.survey_lists[0][0].calc_reward_function(conditions)

In [ ]:
scheduler.survey_lists[0][0].generate_observations(conditions)

In [ ]:
scheduler.survey_lists[0][0]._check_feasibility(conditions)

In [ ]:
scheduler.survey_lists[0][0].indx

In [ ]:
scheduler.survey_lists[0][0].basis_weights

In [ ]:
import healpy as hp

In [ ]:
np.unique(scheduler.survey_lists[0][0].basis_functions[0](conditions))

In [ ]:
hp.mollview(scheduler.survey_lists[0][0].basis_functions[1](conditions))

In [ ]:
scheduler.survey_lists[0][0].basis_functions[1](conditions)[scheduler.survey_lists[0][0].indx]

In [ ]:
reward = 0
for bf, weight in zip(scheduler.survey_lists[0][0].basis_functions, scheduler.survey_lists[0][0].basis_weights):
    basis_value = bf(conditions, indx=scheduler.survey_lists[0][0].indx)
    reward += basis_value * weight

In [ ]:
hp.mollview(reward)

In [ ]:
reward[scheduler.survey_lists[0][0].indx]

In [ ]:
if not np.isscalar(reward):
    reward = np.sum(reward[scheduler.survey_lists[0][0].indx])

In [ ]:
2 * reward

In [ ]:
dir(scheduler.survey_lists[0][0])

In [ ]:
conditions.mounted_filters

In [ ]:
print(conditions.tel_alt)
print(conditions.tel_az)
print(conditions.tel_ra)
print(conditions.tel_dec)

In [ ]:
from astroplan.plots import plot_airmass
from astroplan import Observer, FixedTarget
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt

In [ ]:
observer = Observer.at_site("LSST")

In [ ]:
time = observer.midnight(time=Time('2024-11-04 00:00:00.000', format='iso'), which='nearest')

In [ ]:
observer.twilight_evening_astronomical(time=Time('2024-11-04 00:00:00.000', format='iso'), which='nearest').iso

In [ ]:
dir(observer)

In [ ]:
ra = 30.0
dec = -60.0
name = 'target'
targets = [
    FixedTarget(SkyCoord(ra, dec, unit='deg', frame='icrs'), name=name)
]

In [ ]:
plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
def get_scheduler():
    nside = 32
    #per_night = True  # Dither DDF per night
    #detailer = detailers.DitherDetailer(max_dither=0.2, per_night=False)
    #n_offsets = 30
    #night = 42
    #offsets = np.degrees(detailer._generate_offsets(n_offsets, night))
    camera_ddf_rot_limit = 75.0
    # Existing
    #observatory = ModelObservatory(nside=nside, mjd_start=MJD_START)

    # New
    filter_scheduler = ComCamFilterSched(
        loaded_filter_groups=(('g', 'r', 'i'),),
        illum_bins=(0, 100)
    )
    dayobs = '20241103'
    observatory = get_model_observatory(dayobs)
    observatory = update_model_observatory_sunset(observatory, filter_scheduler)

    
    observatory.sky_model.load_length = 3
    #observatory = update_model_observatory_sunset(observatory, filter_scheduler)
    conditions = observatory.return_conditions()
    sky = SkyAreaGenerator(nside=nside)
    footprints_hp, footprints_labels = sky.return_maps()
    footprints = Footprint(
        MJD_START,
        sun_ra_start=conditions.sun_ra,
        nside=nside,
        filters=["u", "g", "r_03", "i_06", "z", "y"],
    )
    for i, key in enumerate(footprints_hp.dtype.names):
        footprints.footprints[i, :] = footprints_hp[key]
    footprints.filters = dict(
        u=0,
        g=1,
        r_03=2,
        i_06=3,
        z=4,
        y=5,
    )
    filters_list = [
        "i_06",
        "r_03"
    ]
    surveys = []
    i = 0
    for filter in filters_list:
        for offset in range(0, 31):
            surveys.append(gen_field_survey(f'BLOCK-T248_{i}'), sequence=[filter])
            i += 1
    scheduler = CoreScheduler(surveys, nside=nside)

    return nside, scheduler

In [ ]:
nside, scheduler = get_scheduler()